In the previous Notebooks i've established that using a Linear regression to predict Rank is my best bet.

This method is called a pointwise ranking method. and the "learn to rank" Theory proposes more ranking methods that i'll try to explore in this norebook. 

    -Pairwise ranking 
    -Listwise ranking

Let's start with Pairwise ranking there are various methods all from Microsoft researcher Burges
RankNet, LambdaRank, LambdaRankMart

In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')
Y_train['Rank']= Y_train['TARGET'].rank()
Y_train

,ID,TARGET,Rank
0,1054,0.028313,807.0
1,2049,-0.112516,532.0
2,1924,-0.180840,427.0
3,297,-0.260356,343.0
4,1101,-0.071733,593.0
...,...,...,...
1489,459,-0.172597,440.0
1490,1674,-0.063546,609.0
1491,748,0.151797,1016.0
1492,1454,-0.640917,163.0


In [3]:
df=create_pairwise_dataset(Y_train)
print(df)
remade=reconstruct_original_dataset(df)
remade

            ID1     ID2  Comparison
0        1054.0  2049.0         1.0
1        1054.0  1924.0         1.0
2        1054.0   297.0         1.0
3        1054.0  1101.0         1.0
4        1054.0  1520.0         0.0
...         ...     ...         ...
1115266  1674.0  1454.0         1.0
1115267  1674.0   359.0         1.0
1115268   748.0  1454.0         1.0
1115269   748.0   359.0         1.0
1115270  1454.0   359.0         0.0

[1115271 rows x 3 columns]


,ID,Rank_recreated
0,0.0,455
1,2.0,778
2,3.0,364
3,4.0,810
4,5.0,589
...,...,...
1489,2142.0,1357
1490,2143.0,1314
1491,2144.0,571
1492,2145.0,434


1115271


1115271.0

In [4]:
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
sorted['ID'].tolist()==Y_train['ID'].tolist()

True

In [6]:
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_train["TARGET"]).correlation)


TARGET AND reconstructed Rank : 1.0


Predicting Pairwise ranking accurately -> predicting Ranking accurately -> high spearman score

meaning our method of creating pairwise ranking and prediction a pairwise ranking ID by ID might help determine a more accurate ranking. Now let's Get a  X_train_pairwise that matches our Y_train_Pairwise

In [7]:
X_train_pairwise=get_training_pairwise(X_train)

In [2]:


from sklearn.linear_model import LinearRegression
from utils.compute_model_performance import *

model=Model(LinearRegression,LinearRegression)
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=5,columns='Rank_recreated',split=False)



X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
error -0.008192940778567982
accuracy 0.5577841177904259
reconstructing rank ...
Y_pred and Y_test same ID ? True
Same length ? True
         ID  Rank_recreated    ID    TARGET    Rank
0    1099.0             222  1099 -0.030868   669.0
1     718.0              72   718 -0.649239   161.0
2     436.0             276   436 -1.750309    39.0
3     337.0             286   337  0.153502  1020.0
4    2017.0             369  2017  1.472587  1402.0
..      ...             ...   ...       ...     ...
369   390.0              72   390 -0.167641   445.0
370  2027.0             369  2027  1.800264  1429.0
371    33.0             371    33 -1.851949    35.0
372  1379.0              72  1379  0.048142   853.0
373   477.0             369   477 -0.389972   261.0

[374 rows x 5 columns]
validation 12.676044075136137
X_train and Y_train same ID ? True
X_test and Y_test same ID ? Tr

In [3]:
from mord import OrdinalRidge
from utils.compute_model_performance import *
model=Model(OrdinalRidge,OrdinalRidge)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=2,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


error -0.05519634127109289
accuracy 0.5450961276540838
reconstructing rank ...
Y_pred and Y_test same ID ? True
Same length ? True
         ID  Rank_recreated    ID    TARGET    Rank
0    1631.0             324  1631  0.290948  1136.0
1     283.0             324   283 -0.025477   684.0
2    2129.0             202  2129  1.327686  1388.0
3     520.0             324   520 -0.160823   458.0
4     344.0             329   344  0.095579   928.0
..      ...             ...   ...       ...     ...
369  1773.0             371  1773 -0.076641   585.0
370  1308.0             371  1308  0.047840   851.0
371  1936.0             371  1936 -0.140645   484.0
372   594.0             371   594 -0.247198   356.0
373    49.0             371    49  0.190419  1054.0

[374 rows x 5 columns]
validation 2.309225783460685
X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


error 0.036071167438459666
accuracy 0.560063655001362
reconstructing rank ...
Y_pred and Y_test same ID ? True
Same length ? True
         ID  Rank_recreated    ID    TARGET    Rank
0     546.0             295   546 -0.086572   568.0
1      79.0             341    79 -0.072683   591.0
2    2013.0             162  2013  0.210736  1081.0
3     136.0             373   136  1.007261  1363.0
4     473.0             153   473 -0.236414   361.0
..      ...             ...   ...       ...     ...
369  1566.0               1  1566  0.540126  1270.0
370   215.0             373   215 -0.145708   473.0
371   718.0             373   718 -0.649239   161.0
372    50.0             373    50 -0.549756   190.0
373   553.0             373   553 -0.035453   659.0

[374 rows x 5 columns]
validation -3.5860170567117153
average Rank_recreated evaluation score at  -0.6383956366255152 training at : nan training at : nan training at : nan




In [3]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from utils.compute_model_performance import *

# model=Model(RandomForestRegressor,RandomForestRegressor,depth=3,n_estimators=10)
# model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=1,columns='Rank_recreated',split=False)

model=Model(RandomForestClassifier,RandomForestClassifier,depth=10,n_estimators=100)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
error_val -0.08096658112428495
accuracy_val 0.5668592564981147
error_train -0.03414959147197753
accuracy_train 0.8115297459466361
reconstructing rank ...
Y_pred and Y_test same ID ? True
Same length ? True
         ID  Rank_recreated    ID    TARGET    Rank
0    1377.0             351  1377  3.726678  1471.0
1    1869.0              80  1869 -0.047611   639.0
2    2061.0             351  2061 -0.071935   592.0
3     279.0             351   279  0.482113  1253.0
4     459.0             352   459 -0.172597   440.0
..      ...             ...   ...       ...     ...
369   224.0             368   224  0.738717  1318.0
370  1930.0             368  1930  0.000000   751.5
371   998.0             368   998 -0.694801   151.0
372  1384.0             368  1384 -0.469155   221.0
373  1399.0             351  1399  0.194651  1061.0

[374 rows x 5 columns]
validation -0.0141170

In [4]:
import xgboost as xgb

model=Model(xgb.XGBClassifier,xgb.XGBClassifier,depth=5,n_estimators=50)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
error_val -0.004358360453613568
accuracy_val 0.5718484322805408
error_train 0.006940986850504277
accuracy_train 0.847665326184093
reconstructing rank ...
Y_pred and Y_test same ID ? True
Same length ? True
         ID  Rank_recreated    ID    TARGET    Rank
0     806.0             243   806  1.860895  1431.0
1     470.0             373   470 -0.815559   120.0
2     166.0             318   166  0.484465  1254.0
3     424.0             370   424  1.292451  1384.0
4     797.0             373   797  0.216824  1085.0
..      ...             ...   ...       ...     ...
369  2013.0             373  2013  0.210736  1081.0
370  1836.0             373  1836  2.358063  1445.0
371  2141.0             373  2141  0.117021   973.0
372  1321.0             373  1321  0.229969  1094.0
373   113.0             373   113  0.572854  1283.0

[374 rows x 5 columns]
validation -8.4608034